In [13]:
# Open file and read all the data into a list
with open('file.txt','r') as f:
    lines = f.readlines()
    lines = [line.replace('\n','') for line in lines]      # Remove newline characters

# Filter all empty strings
lines = list(filter(lambda x: x!='', lines))    
print(lines)

['a,c,d,e,g,h,k', 'k+l+m=>i', 'i+l+j=>q', 'c+d+e=>b', 'a+b=>q', 'l+n+o+p=>q', 'c+h=>r', 'r+j+m=>s', 'f+h=>b', 'g=>f', 'q']


In [14]:
# Read all antecedents into the knowledge base
knowledge_base = set([kn for kn in lines[0].split(',')])
print(knowledge_base)

{'c', 'a', 'k', 'e', 'g', 'd', 'h'}


In [15]:
# Rule base creation
rule_base_rev = {tuple(rule_line.split('=>')[0].split('+')):rule_line.split('=>')[1] for rule_line in lines[1:-1]}  # Split first the premise based on '=>' and then the antecedents based on '+'

# Reverse the rule base (for easier access in backward chaining)
rule_base = {val:[] for val in rule_base_rev.values()}
for key,val in rule_base_rev.items():
    rule_base[val].append(key)
print(rule_base_rev)
print(rule_base)

{('k', 'l', 'm'): 'i', ('i', 'l', 'j'): 'q', ('c', 'd', 'e'): 'b', ('a', 'b'): 'q', ('l', 'n', 'o', 'p'): 'q', ('c', 'h'): 'r', ('r', 'j', 'm'): 's', ('f', 'h'): 'b', ('g',): 'f'}
{'i': [('k', 'l', 'm')], 'q': [('i', 'l', 'j'), ('a', 'b'), ('l', 'n', 'o', 'p')], 'b': [('c', 'd', 'e'), ('f', 'h')], 'r': [('c', 'h')], 's': [('r', 'j', 'm')], 'f': [('g',)]}


In [16]:
# Extract the goal variable
goal_var = lines[-1]
print(goal_var)

q


In [17]:
# Forward Chaining
def forward_chaining(knowledge_base, rule_base):
    new_knowledge_base = knowledge_base.copy()    # Create a copy of the knowledge base to update with new rules
    prev_len = len(new_knowledge_base)            
    while True:
        # Iterate over the rule base each time to see if you gain any new insights(conclusion)
        for conclusion,premises_list in rule_base.items():
            for premises in premises_list:
                if all([premise in new_knowledge_base for premise in premises]) :
                    new_knowledge_base.update(conclusion)

        # If no new conclusion is added, break
        if prev_len == len(new_knowledge_base):
            break
        prev_len = len(new_knowledge_base)
    return new_knowledge_base

print('Initial Knowledge:', knowledge_base)
print('Final Knowledge:', forward_chaining(knowledge_base, rule_base))
print('Knowledge Added:', forward_chaining(knowledge_base, rule_base) - knowledge_base)

Initial Knowledge: {'c', 'a', 'k', 'e', 'g', 'd', 'h'}
Final Knowledge: {'c', 'q', 'r', 'b', 'k', 'f', 'd', 'h', 'a', 'e', 'g'}
Knowledge Added: {'f', 'q', 'r', 'b'}


In [20]:
# Backward chaining
def backward_chaining(knowledge_base, rule_base, goal_var):
    # Globals to trace path. 
    global paths # Paths variable will have all the existing antecedents in the knowledge base and the path backward from goal state.
    global initial_goal_state
    
    # Return path if goal is already in the knowledge base
    if goal_var in knowledge_base:
        # If goal variable is already in the knowledge base add it to the path
        if len(paths)==0:
            paths.append([goal_var])
        else:
            paths[-1].append(goal_var)
        return True
    
    # If goal is in the rule base then recurse backward for each antecedent in the rule base and perform backward chaining for each one
    if goal_var in rule_base:
        for premises in rule_base[goal_var]:
            if goal_var==initial_goal_state:
                # Add new list for each route back from goal state to store new paths
                paths.append([])
            all_satisfied=True
            for premise in premises:
                if not backward_chaining(knowledge_base, rule_base, premise):
                    all_satisfied=False
                    break
            if all_satisfied:
                # Update knowledge base and latest path list if each antecedent is found to be a valid conclusion from recursive backward chaining
                knowledge_base.update(goal_var)
                paths[-1].append(str(premises)+'=>'+goal_var)
                return True
                
    return False

paths = []
initial_goal_state = goal_var
res = backward_chaining(knowledge_base.copy(), rule_base, goal_var)
# if goal is a valid conclusion, print the valid path that exists
if res:
    print('Goal state is a valid conclusion. Proved Path:', paths[-1])
else:
    print('Goal state is not a valid conclusion.')

Goal state is a valid conclusion. Proved Path: ['a', 'c', 'd', 'e', "('c', 'd', 'e')=>b", "('a', 'b')=>q"]


In [22]:
res = backward_chaining(knowledge_base.copy(), rule_base, 'l')     # Check for different goal state
if res:
    print('Goal state is a valid conclusion. Proved Path:', paths[-1])
else:
    print('Goal state is not a valid conclusion.')

Goal state is not a valid conclusion.
